In [1]:
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

from collections import defaultdict
from scipy.stats.stats import pearsonr

In [2]:
df=pd.read_csv("/home/umair/Desktop/Data Science and BI/data mining/occupancy_data/datatest2.csv")


In [3]:
df.head()

,id,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
0,1,11/02/15 14:48,21.7600,31.133333,437.333333,1029.666667,0.005021,1
1,2,11/02/15 14:49,21.7900,31.000000,437.333333,1000.000000,0.005009,1
2,3,11/02/15 14:50,21.7675,31.122500,434.000000,1003.750000,0.005022,1
3,4,11/02/15 14:51,21.7675,31.122500,439.000000,1009.500000,0.005022,1
4,5,11/02/15 14:51,21.7900,31.133333,437.333333,1005.666667,0.005030,1


In [4]:
df.isnull().sum()

id               0
date             0
Temperature      0
Humidity         0
Light            0
CO2              0
HumidityRatio    0
Occupancy        0
dtype: int64

In [5]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

,id,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
id,1,-0.190574,0.269044,-0.0488875,0.68834,0.227989,0.0309893
Temperature,-0.190574,1,-0.472921,0.705538,0.199646,-0.0104072,0.521767
Humidity,0.269044,-0.472921,1,-0.187477,-0.0792236,0.884104,-0.0877677
Light,-0.0488875,0.705538,-0.187477,1,0.190213,0.162681,0.914525
CO2,0.68834,0.199646,-0.0792236,0.190213,1,0.0431972,0.261908
HumidityRatio,0.227989,-0.0104072,0.884104,0.162681,0.0431972,1,0.190161
Occupancy,0.0309893,0.521767,-0.0877677,0.914525,0.261908,0.190161,1


In [6]:
df['Occupancy'].value_counts()

0    7703
1    2049
Name: Occupancy, dtype: int64

In [7]:
#Change all the non zero features in 1
df.loc[df['Light'] < 370, 'Light'] = 0
df.loc[df['Light'] >= 370, 'Light'] = 1

In [8]:
df['Light'].value_counts()

0.0    7661
1.0    2091
Name: Light, dtype: int64

In [9]:
df['Occupancy'].value_counts()

0    7703
1    2049
Name: Occupancy, dtype: int64

In [10]:
#Crossatab between Occupancy and light
pclass_xt = pd.crosstab(df['Light'], df['Occupancy'])
pclass_xt

Occupancy,0,1
Light,,
0.0,7649,12
1.0,54,2037


In [11]:
#Create a new dataframe with only the division of date and hour
df1=pd.DataFrame({
    'date': pd.to_datetime(df['date']).dt.date,
    'time': pd.to_datetime(df['date']).dt.hour})

In [12]:
#Adding teh previous dataframe to the main dataframe
df['date']=df1['date']
df['Hour']=df1['time']

In [13]:
#The new dataframe with date and time splitted
df.head()

,id,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy,Hour
0,1,2015-11-02,21.7600,31.133333,1.0,1029.666667,0.005021,1,14
1,2,2015-11-02,21.7900,31.000000,1.0,1000.000000,0.005009,1,14
2,3,2015-11-02,21.7675,31.122500,1.0,1003.750000,0.005022,1,14
3,4,2015-11-02,21.7675,31.122500,1.0,1009.500000,0.005022,1,14
4,5,2015-11-02,21.7900,31.133333,1.0,1005.666667,0.005030,1,14


In [14]:
#Adjusting the order of the column
df=df[['id', 'date', 'Hour', 'Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio', 'Occupancy']]

In [15]:
df.head()

,id,date,Hour,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
0,1,2015-11-02,14,21.7600,31.133333,1.0,1029.666667,0.005021,1
1,2,2015-11-02,14,21.7900,31.000000,1.0,1000.000000,0.005009,1
2,3,2015-11-02,14,21.7675,31.122500,1.0,1003.750000,0.005022,1
3,4,2015-11-02,14,21.7675,31.122500,1.0,1009.500000,0.005022,1
4,5,2015-11-02,14,21.7900,31.133333,1.0,1005.666667,0.005030,1


In [16]:
#Add the column with the name of the day
df['date'] = pd.to_datetime(df['date'], exact=False, format='%Y-%d-%m')

df['DayName'] = df['date'].dt.weekday_name

In [17]:
#Adjusting the order of the column
df=df[['id', 'date', 'DayName', 'Hour', 'Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio', 'Occupancy']]

In [18]:
df.head()

,id,date,DayName,Hour,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
0,1,2015-02-11,Wednesday,14,21.7600,31.133333,1.0,1029.666667,0.005021,1
1,2,2015-02-11,Wednesday,14,21.7900,31.000000,1.0,1000.000000,0.005009,1
2,3,2015-02-11,Wednesday,14,21.7675,31.122500,1.0,1003.750000,0.005022,1
3,4,2015-02-11,Wednesday,14,21.7675,31.122500,1.0,1009.500000,0.005022,1
4,5,2015-02-11,Wednesday,14,21.7900,31.133333,1.0,1005.666667,0.005030,1


In [19]:
#Add a new column for the boolean variable IsWorkDay, this feature is 1 if the day is a work day, 0 otherwise
df["IsWorkDay"] = np.nan
df['IsWorkDay']=df['IsWorkDay'].astype(bool).astype(int)

In [20]:
#Fill the feature IsWorkDay with the corrsponding 1 or 0
for index, row in df.iterrows():
    if row["DayName"]!="Saturday" and row["DayName"]!="Sunday":
        df.loc[index, "IsWorkDay"] = 1
    else:
        df.loc[index, "IsWorkDay"] = 0

In [21]:
#Adjusting the order of the column
df=df[['id', 'date', 'DayName', 'IsWorkDay', 'Hour', 'Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio', 'Occupancy']]

In [22]:
#Drop the column id because is useless for us
df.drop(['id'], axis=1, inplace =True)

In [23]:
#Drop the column HumidityRatio because is useless for us
df.drop(['HumidityRatio'], axis=1, inplace =True)

In [24]:
#Final attributes of our cleaned dataset
df.head()

,date,DayName,IsWorkDay,Hour,Temperature,Humidity,Light,CO2,Occupancy
0,2015-02-11,Wednesday,1,14,21.7600,31.133333,1.0,1029.666667,1
1,2015-02-11,Wednesday,1,14,21.7900,31.000000,1.0,1000.000000,1
2,2015-02-11,Wednesday,1,14,21.7675,31.122500,1.0,1003.750000,1
3,2015-02-11,Wednesday,1,14,21.7675,31.122500,1.0,1009.500000,1
4,2015-02-11,Wednesday,1,14,21.7900,31.133333,1.0,1005.666667,1


In [25]:
#Add a new column for the variable IsWorkHour, this feature is 1 if the hour is a work hour,
#0 if it isn't a work hour and 2 if we have the doubt, so in the boundary values 
df["IsWorkHour"] = np.nan
df['IsWorkHour']=df['IsWorkHour'].astype(bool).astype(int)

In [26]:
#Fill the feature IsWorkHour with the corrsponding 0, 1 or 2
for index, row in df.iterrows():
    if row["Hour"]==13 or row["Hour"]==7 or row["Hour"]==18:
        df.loc[index, "IsWorkHour"] = 2 #could be both a work hour or not
    elif row["Hour"]<7 or row["Hour"]>18:
        df.loc[index, "IsWorkHour"] = 0 #is not a work hour
    else:
        df.loc[index, "IsWorkHour"] = 1 #is a work hour

In [27]:
#Adjusting the order of the column
df=df[['date', 'DayName', 'IsWorkDay', 'Hour', 'IsWorkHour', 'Temperature', 'Humidity', 'Light', 'CO2', 'Occupancy']]

In [28]:
#test for see if everything works
df[1300:3000]

,date,DayName,IsWorkDay,Hour,IsWorkHour,Temperature,Humidity,Light,CO2,Occupancy
1300,2015-02-12,Thursday,1,12,1,24.166667,22.700000,0.0,726.000000,0
1301,2015-02-12,Thursday,1,12,1,24.200000,22.700000,0.0,729.000000,0
1302,2015-02-12,Thursday,1,12,1,24.200000,22.700000,0.0,729.750000,0
1303,2015-02-12,Thursday,1,12,1,24.200000,22.700000,0.0,727.750000,0
1304,2015-02-12,Thursday,1,12,1,24.200000,22.700000,0.0,724.000000,0
...,...,...,...,...,...,...,...,...,...,...
2995,2015-01-02,Friday,1,16,1,21.760000,31.290000,1.0,608.000000,1
2996,2015-01-02,Friday,1,16,1,21.700000,31.315000,1.0,610.750000,1
2997,2015-01-02,Friday,1,16,1,21.700000,31.290000,1.0,612.500000,1
2998,2015-01-02,Friday,1,16,1,21.700000,31.356667,1.0,617.333333,1


In [29]:
#Export the dataset cleaned
df.to_csv(r'DataCleanedTest2.csv')

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9752 entries, 0 to 9751
Data columns (total 10 columns):
date           9752 non-null datetime64[ns]
DayName        9752 non-null object
IsWorkDay      9752 non-null int64
Hour           9752 non-null int64
IsWorkHour     9752 non-null int64
Temperature    9752 non-null float64
Humidity       9752 non-null float64
Light          9752 non-null float64
CO2            9752 non-null float64
Occupancy      9752 non-null int64
dtypes: datetime64[ns](1), float64(4), int64(4), object(1)
memory usage: 762.0+ KB


In [31]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

/opt/anaconda3/lib/python3.7/site-packages/matplotlib/colors.py:527: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


,IsWorkDay,Hour,IsWorkHour,Temperature,Humidity,Light,CO2,Occupancy
IsWorkDay,nan,nan,nan,nan,nan,nan,nan,nan
Hour,nan,1,0.133651,0.192461,0.221077,0.122788,-0.000385104,0.124978
IsWorkHour,nan,0.133651,1,0.444041,-0.0177979,0.354939,0.073208,0.347668
Temperature,nan,0.192461,0.444041,1,-0.472921,0.528576,0.199646,0.521767
Humidity,nan,0.221077,-0.0177979,-0.472921,1,-0.0938509,-0.0792236,-0.0877677
Light,nan,0.122788,0.354939,0.528576,-0.0938509,1,0.258453,0.979844
CO2,nan,-0.000385104,0.073208,0.199646,-0.0792236,0.258453,1,0.261908
Occupancy,nan,0.124978,0.347668,0.521767,-0.0877677,0.979844,0.261908,1
